In [1]:
import pandas as pd

# Carregar os dados da planilha
file_path = r'C:\Users\MatheusBarros\OneDrive - SDG Group\Documentos\Github\case_infineat\src\Planilha_case_dados.xlsx'
data_plan1 = pd.read_excel(file_path, sheet_name='Plan1')
data_plan3 = pd.read_excel(file_path, sheet_name='Plan3')


data_plan3.columns = data_plan1.columns
data_plan1 = pd.concat([data_plan1,data_plan3])

In [17]:
import pandas as pd

# Carregar os dados da planilha
file_path = r'C:\Users\MatheusBarros\OneDrive - SDG Group\Documentos\Github\case_infineat\src\Planilha_case_dados.xlsx'
data_plan1 = pd.read_excel(file_path, sheet_name='Plan1')
data_plan3 = pd.read_excel(file_path, sheet_name='Plan3')

data_plan1['UF - LOJA'].str.split('-').str[0].str.strip()
# data_plan1['UF - LOJA'].str.split('(').str[1].str.replace(')','').str.strip()


0       DF
1       DF
2       DF
3       DF
4       SP
        ..
1401    MG
1402    ES
1403    ES
1404    ES
1405    ES
Name: UF - LOJA, Length: 1406, dtype: object

In [8]:
# Verificar e renomear colunas para padronização (caso necessário)
data_plan1.columns = [col.strip().lower().replace(' ', '_') for col in data_plan1.columns]

# Remover entradas com valores inválidos (0 em valor total ou pesagem total)
data_plan1 = data_plan1[(data_plan1['valor_total_geral_(r$)'] > 0) & (data_plan1['pesagem_real_total_(kg)'] > 0)]

# Cálculo da eficiência por fornecedor (custo por kg descartado)
data_plan1['custo_por_kg'] = data_plan1['valor_total_geral_(r$)'] / data_plan1['pesagem_real_total_(kg)']

In [9]:
# Agrupar por fornecedor para calcular a média de custo por kg, peso total e eficiência
fornecedor_efficiency = data_plan1.groupby('fornecedor').agg(
    peso_total=('pesagem_real_total_(kg)', 'sum'),
    valor_total=('valor_total_geral_(r$)', 'sum'),
    custo_medio_por_kg=('custo_por_kg', 'mean'),
    num_transacoes=('fornecedor', 'count')
).reset_index()

# Calcular a métrica de eficiência geral (custo médio ponderado pelo peso total)
fornecedor_efficiency['eficiencia_geral'] = fornecedor_efficiency['valor_total'] / fornecedor_efficiency['peso_total']

# Ordenar pela eficiência (menor custo médio por kg e menor eficiência geral)
fornecedor_efficiency = fornecedor_efficiency.sort_values(by=['eficiencia_geral', 'custo_medio_por_kg'])

# Adicionar um ranking de eficiência
fornecedor_efficiency['ranking'] = fornecedor_efficiency['custo_medio_por_kg'].rank(method='min')

# Ordenar o DataFrame pelo ranking
fornecedor_efficiency = fornecedor_efficiency.sort_values(by='ranking')

fornecedor_efficiency


,fornecedor,peso_total,valor_total,custo_medio_por_kg,num_transacoes,eficiencia_geral,ranking
23,Recol,773000,40814.4000,0.052800,15,0.052800,1.0
21,Ouro Verde,139680,11872.8000,0.085000,83,0.085000,2.0
7,COPEL,77740,6930.0000,0.106182,11,0.089143,3.0
12,ELLUS,48220,5966.4430,0.123973,6,0.123734,4.0
20,MULTFACCE,24800,4651.1200,0.164043,25,0.187545,5.0
19,MARCA CONSTRUTORA,35670,5987.4100,0.171088,4,0.167856,6.0
6,CONSTRURBAN,9620,1950.0000,0.202703,1,0.202703,7.0
4,CAPITAL RECICLAVEIS,147695,17400.0000,0.203733,87,0.117810,8.0
3,BRASLIMP,63000,14441.3000,0.229227,28,0.229227,9.0
8,CORPUS,42700,10197.8000,0.238774,48,0.238824,10.0


In [10]:
# Ranking de lojas por custo por kg
data_plan1['uf_-_loja'] = data_plan1['uf_-_loja'].fillna('Desconhecida')  # Tratar lojas com valores nulos
loja_ranking = data_plan1.groupby('uf_-_loja').agg(
    peso_total=('pesagem_real_total_(kg)', 'sum'),
    valor_total=('valor_total_geral_(r$)', 'sum'),
    custo_medio_por_kg=('custo_por_kg', 'mean')
).reset_index()

# Ordenar por maior custo médio por kg
loja_ranking = loja_ranking.sort_values(by='custo_medio_por_kg', ascending=False)

# Salvar os resultados em arquivos CSV para futuras análises
loja_ranking


,uf_-_loja,peso_total,valor_total,custo_medio_por_kg
53,MG - BAIRRO LAGOA SANTA (MLS),346,3829.72,12.903727
55,MG - BAIRRO SION II ALASKA (MAK),322,2261.34,7.496251
66,MG - PAMPULHA (MPA),375,1507.56,4.150833
52,MG - AMAZONAS (MAZ),2885,6467.95,3.868663
104,SP - BAIRRO RIBEIRAO PRETO (CRB),590,2200.99,3.730492
...,...,...,...,...
74,Morada do Sol,60000,3168.00,0.052800
37,Garavelo,53000,2798.40,0.052800
39,Goiânia Shopping,40000,2112.00,0.052800
49,Laranjeiras,77000,4065.60,0.052800


In [11]:
# Exemplo de simulação de redução
percentuais = [10, 20, 30]
resultados_simulacao = []

for percentual in percentuais:
    reducao_kg = loja_ranking['peso_total'] * (percentual / 100)
    custo_reduzido = loja_ranking['custo_medio_por_kg'] * reducao_kg
    economias = loja_ranking['valor_total'] - custo_reduzido
    resultados_simulacao.append((percentual, economias.sum()))

# Mostrar resultados
for percentual, economia in resultados_simulacao:
    print(f"Economia com redução de {percentual}%: R$ {economia:.2f}")


Economia com redução de 10%: R$ 737315.93
Economia com redução de 20%: R$ 642735.95
Economia com redução de 30%: R$ 548155.97
